In [1]:
from pysheds.grid import Grid
from skimage.morphology import skeletonize
import matplotlib.pyplot as plt
from skimage.io import imread
from random import randrange
import numpy as np
import cv2
from PIL import Image
import os
import h5py

In [2]:
# Parameters to change !!!

sketch_rivers_flag = True
sketch_basins_flag = True
sketch_ridges_flag = True
sketch_peaks_flag = True

imageDimensionX = 512
imageDimensionY = 512

# last tile nr
iterateFrom = 32000

colorRivers = (0, 255, 255)
colorBasins = (0, 255, 255)

colorRidges = (250, 212, 250)
colorPeaks = (250, 212, 250)

maxReadImages = 2000

savePathDestination = 'images/train512_55/terrain_'
readDEMPath = 'heightmapsDEM_Kalahari_512/topography_'
readTIFFPath = 'heightmapsTIFF_Kalahari_512/topography_'

In [3]:
def sketch_rivers(tiff_image):
    grid = Grid.from_raster(tiff_image, data_name='dem')
    grid.fill_depressions(data='dem', out_name='flooded_dem')
    depressions =  grid.detect_depressions('flooded_dem')
    grid.resolve_flats(data='flooded_dem', out_name='inflated_dem')

    dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
    grid.flowdir(data='inflated_dem', out_name='dir', dirmap=dirmap)
    grid.accumulation(data='dir', out_name='acc', dirmap=dirmap)
    
    upsampled_rivers = np.log(grid.view('acc') + 1)
    upsampled_rivers = (upsampled_rivers - np.amin(upsampled_rivers)) / (np.amax(upsampled_rivers) - np.amin(upsampled_rivers))
    upsampled_rivers = np.array(upsampled_rivers * 255, dtype=np.uint8)
    _, thresholded_river = cv2.threshold(upsampled_rivers, 190, 255, cv2.THRESH_BINARY)
    thresholded_river[thresholded_river == 255] = 1
    skeletonized_rivers = skeletonize(thresholded_river)

    rivers_sketch = Image.fromarray(skeletonized_rivers)
    rgb_rivers_sketch = rivers_sketch.convert('RGB')
    
    basins_sketch = Image.fromarray(depressions)
    rgb_basins_sketch = basins_sketch.convert('RGB')
    
    return rgb_rivers_sketch, rgb_basins_sketch

In [4]:
def sketch_ridges(tiff_image):
    grid = Grid.from_raster(tiff_image, data_name='dem')
    grid.dem = grid.dem.max() - grid.dem
    grid.fill_depressions(data='dem', out_name='flooded_dem')
    peaks = grid.detect_depressions('flooded_dem')
    
    grid.resolve_flats(data='flooded_dem', out_name='inflated_dem')
    dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
    grid.flowdir(data='inflated_dem', out_name='dir', dirmap=dirmap)
    grid.accumulation(data='dir', out_name='acc', dirmap=dirmap)

    upsampled_ridges = np.log(grid.view('acc') + 1)
    upsampled_ridges = (upsampled_ridges - np.amin(upsampled_ridges)) / (np.amax(upsampled_ridges) - np.amin(upsampled_ridges))
    upsampled_ridges = np.array(upsampled_ridges * 255, dtype=np.uint8)
    _, thresholded_ridges = cv2.threshold(upsampled_ridges, 190, 255, cv2.THRESH_BINARY)
    thresholded_ridges[thresholded_ridges == 255] = 1
    skeletonized_ridges = skeletonize(thresholded_ridges)
    ridges_sketch = Image.fromarray(skeletonized_ridges)
    rgb_ridges_sketch = ridges_sketch.convert('RGB')
    
    peaks_sketch = Image.fromarray(peaks)
    rgb_peaks_sketch = peaks_sketch.convert('RGB')
    
    return rgb_ridges_sketch, rgb_peaks_sketch

In [5]:
def prepare_heightmap_sketch(file_path):
    rivers, basins = sketch_rivers(file_path)
    ridges, peaks = sketch_ridges(file_path)

    sketch = np.zeros([imageDimensionX,imageDimensionY,3],dtype=np.uint8)
    sketch.fill(255)
        
    random = randrange(3)
    
    if random == 0:
        sketch_rivers_flag = True
        sketch_basins_flag = True
        sketch_ridges_flag = False
        sketch_peaks_flag = False
    elif random == 1:
        sketch_rivers_flag = False
        sketch_basins_flag = False
        sketch_ridges_flag = True
        sketch_peaks_flag = True
    else:
        sketch_rivers_flag = True
        sketch_basins_flag = True
        sketch_ridges_flag = True
        sketch_peaks_flag = True
        
        
    if sketch_rivers_flag == True:
        pixels_rivers = rivers.load()
  
        for i in range(rivers.size[0]):
            for j in range(rivers.size[1]):
                if pixels_rivers[i,j] == (255, 255, 255):
                    sketch[j,i] = colorRivers
    
    
    if sketch_ridges_flag == True:
        pixels_ridges = ridges.load()
            
        for i in range(ridges.size[0]):
            for j in range(ridges.size[1]):
                if pixels_ridges[i,j] == (255, 255, 255):
                    sketch[j,i] = colorRidges 
    
    
    if sketch_peaks_flag == True:
        pixels_peaks = peaks.load()

        for i in range(peaks.size[0]):
            for j in range(peaks.size[1]):
                if pixels_peaks[i,j] == (255, 255, 255):
                    sketch[j,i] = colorPeaks  
            
    
    if sketch_basins_flag == True:
        pixels_basins = basins.load()
            
        for i in range(basins.size[0]):
            for j in range(basins.size[1]):
                if pixels_basins[i,j] == (255, 255, 255):
                    sketch[j,i] = colorBasins

    
    return sketch

In [6]:
files_len = len(os.listdir('heightmapsDEM_Kalahari_512')) * 4
files_len_directory = len(os.listdir('heightmapsDEM_Kalahari_512'))

if files_len > (maxReadImages * 4):
    files_len = maxReadImages * 4
    files_len_directory = maxReadImages

idxs = [x for x in range(files_len)]
np.random.shuffle(idxs)

In [7]:
dataset_iterator = 0
iterator = 0

for iterator in range(files_len_directory):       
    print(str(dataset_iterator + iterateFrom) + '/' + str(files_len + iterateFrom))
    sketch = prepare_heightmap_sketch(readDEMPath + str(iterator+1) + '.tif')
    heightmap = cv2.imread(readTIFFPath + str(iterator+1) + '.tif')
    
    print('topography_' + str(iterator+1) + '.tif')

    sketch_array = np.array(sketch)
    heightmap_array = np.array(heightmap)
    
    im_h = cv2.hconcat([sketch_array, heightmap_array])
    cv2.imwrite(savePathDestination + str((idxs[dataset_iterator]) + iterateFrom) + ".png",im_h)
    dataset_iterator += 1
        
    # rotate90
    sketch_array = np.rot90(sketch_array)
    heightmap_array = np.rot90(heightmap_array)
        
    im_h = cv2.hconcat([sketch_array, heightmap_array])
    cv2.imwrite(savePathDestination + str((idxs[dataset_iterator]) + iterateFrom) + ".png",im_h)
    dataset_iterator += 1
        
    # rotate90
    sketch_array = np.rot90(sketch_array)
    heightmap_array = np.rot90(heightmap_array)
        
    im_h = cv2.hconcat([sketch_array, heightmap_array])
    cv2.imwrite(savePathDestination + str((idxs[dataset_iterator]) + iterateFrom) + ".png",im_h)    
    dataset_iterator += 1
        
    # rotate90
    sketch_array = np.rot90(sketch_array)
    heightmap_array = np.rot90(heightmap_array)
    
    im_h = cv2.hconcat([sketch_array, heightmap_array])
    cv2.imwrite(savePathDestination + str((idxs[dataset_iterator]) + iterateFrom) + ".png",im_h)
    dataset_iterator += 1

print(str(dataset_iterator + iterateFrom) + '/' + str(files_len + iterateFrom))
print("Done!")

32000/40000
topography_1.tif
32004/40000
topography_2.tif
32008/40000
topography_3.tif
32012/40000
topography_4.tif
32016/40000
topography_5.tif
32020/40000
topography_6.tif
32024/40000
topography_7.tif
32028/40000
topography_8.tif
32032/40000
topography_9.tif
32036/40000
topography_10.tif
32040/40000
topography_11.tif
32044/40000
topography_12.tif
32048/40000
topography_13.tif
32052/40000
topography_14.tif
32056/40000
topography_15.tif
32060/40000
topography_16.tif
32064/40000
topography_17.tif
32068/40000
topography_18.tif
32072/40000
topography_19.tif
32076/40000
topography_20.tif
32080/40000
topography_21.tif
32084/40000
topography_22.tif
32088/40000
topography_23.tif
32092/40000
topography_24.tif
32096/40000
topography_25.tif
32100/40000
topography_26.tif
32104/40000
topography_27.tif
32108/40000
topography_28.tif
32112/40000
topography_29.tif
32116/40000
topography_30.tif
32120/40000
topography_31.tif
32124/40000
topography_32.tif
32128/40000
topography_33.tif
32132/40000
topogra

topography_269.tif
33076/40000
topography_270.tif
33080/40000
topography_271.tif
33084/40000
topography_272.tif
33088/40000
topography_273.tif
33092/40000
topography_274.tif
33096/40000
topography_275.tif
33100/40000
topography_276.tif
33104/40000
topography_277.tif
33108/40000
topography_278.tif
33112/40000
topography_279.tif
33116/40000
topography_280.tif
33120/40000
topography_281.tif
33124/40000
topography_282.tif
33128/40000
topography_283.tif
33132/40000
topography_284.tif
33136/40000
topography_285.tif
33140/40000
topography_286.tif
33144/40000
topography_287.tif
33148/40000
topography_288.tif
33152/40000
topography_289.tif
33156/40000
topography_290.tif
33160/40000
topography_291.tif
33164/40000
topography_292.tif
33168/40000
topography_293.tif
33172/40000
topography_294.tif
33176/40000
topography_295.tif
33180/40000
topography_296.tif
33184/40000
topography_297.tif
33188/40000
topography_298.tif
33192/40000
topography_299.tif
33196/40000
topography_300.tif
33200/40000
topograp

topography_534.tif
34136/40000
topography_535.tif
34140/40000
topography_536.tif
34144/40000
topography_537.tif
34148/40000
topography_538.tif
34152/40000
topography_539.tif
34156/40000
topography_540.tif
34160/40000
topography_541.tif
34164/40000
topography_542.tif
34168/40000
topography_543.tif
34172/40000
topography_544.tif
34176/40000
topography_545.tif
34180/40000
topography_546.tif
34184/40000
topography_547.tif
34188/40000
topography_548.tif
34192/40000
topography_549.tif
34196/40000
topography_550.tif
34200/40000
topography_551.tif
34204/40000
topography_552.tif
34208/40000
topography_553.tif
34212/40000
topography_554.tif
34216/40000
topography_555.tif
34220/40000
topography_556.tif
34224/40000
topography_557.tif
34228/40000
topography_558.tif
34232/40000
topography_559.tif
34236/40000
topography_560.tif
34240/40000
topography_561.tif
34244/40000
topography_562.tif
34248/40000
topography_563.tif
34252/40000
topography_564.tif
34256/40000
topography_565.tif
34260/40000
topograp

topography_799.tif
35196/40000
topography_800.tif
35200/40000
topography_801.tif
35204/40000
topography_802.tif
35208/40000
topography_803.tif
35212/40000
topography_804.tif
35216/40000
topography_805.tif
35220/40000
topography_806.tif
35224/40000
topography_807.tif
35228/40000
topography_808.tif
35232/40000
topography_809.tif
35236/40000
topography_810.tif
35240/40000
topography_811.tif
35244/40000
topography_812.tif
35248/40000
topography_813.tif
35252/40000
topography_814.tif
35256/40000
topography_815.tif
35260/40000
topography_816.tif
35264/40000
topography_817.tif
35268/40000
topography_818.tif
35272/40000
topography_819.tif
35276/40000
topography_820.tif
35280/40000
topography_821.tif
35284/40000
topography_822.tif
35288/40000
topography_823.tif
35292/40000
topography_824.tif
35296/40000
topography_825.tif
35300/40000
topography_826.tif
35304/40000
topography_827.tif
35308/40000
topography_828.tif
35312/40000
topography_829.tif
35316/40000
topography_830.tif
35320/40000
topograp

topography_1062.tif
36248/40000
topography_1063.tif
36252/40000
topography_1064.tif
36256/40000
topography_1065.tif
36260/40000
topography_1066.tif
36264/40000
topography_1067.tif
36268/40000
topography_1068.tif
36272/40000
topography_1069.tif
36276/40000
topography_1070.tif
36280/40000
topography_1071.tif
36284/40000
topography_1072.tif
36288/40000
topography_1073.tif
36292/40000
topography_1074.tif
36296/40000
topography_1075.tif
36300/40000
topography_1076.tif
36304/40000
topography_1077.tif
36308/40000
topography_1078.tif
36312/40000
topography_1079.tif
36316/40000
topography_1080.tif
36320/40000
topography_1081.tif
36324/40000
topography_1082.tif
36328/40000
topography_1083.tif
36332/40000
topography_1084.tif
36336/40000
topography_1085.tif
36340/40000
topography_1086.tif
36344/40000
topography_1087.tif
36348/40000
topography_1088.tif
36352/40000
topography_1089.tif
36356/40000
topography_1090.tif
36360/40000
topography_1091.tif
36364/40000
topography_1092.tif
36368/40000
topograp

topography_1319.tif
37276/40000
topography_1320.tif
37280/40000
topography_1321.tif
37284/40000
topography_1322.tif
37288/40000
topography_1323.tif
37292/40000
topography_1324.tif
37296/40000
topography_1325.tif
37300/40000
topography_1326.tif
37304/40000
topography_1327.tif
37308/40000
topography_1328.tif
37312/40000
topography_1329.tif
37316/40000
topography_1330.tif
37320/40000
topography_1331.tif
37324/40000
topography_1332.tif
37328/40000
topography_1333.tif
37332/40000
topography_1334.tif
37336/40000
topography_1335.tif
37340/40000
topography_1336.tif
37344/40000
topography_1337.tif
37348/40000
topography_1338.tif
37352/40000
topography_1339.tif
37356/40000
topography_1340.tif
37360/40000
topography_1341.tif
37364/40000
topography_1342.tif
37368/40000
topography_1343.tif
37372/40000
topography_1344.tif
37376/40000
topography_1345.tif
37380/40000
topography_1346.tif
37384/40000
topography_1347.tif
37388/40000
topography_1348.tif
37392/40000
topography_1349.tif
37396/40000
topograp

topography_1576.tif
38304/40000
topography_1577.tif
38308/40000
topography_1578.tif
38312/40000
topography_1579.tif
38316/40000
topography_1580.tif
38320/40000
topography_1581.tif
38324/40000
topography_1582.tif
38328/40000
topography_1583.tif
38332/40000
topography_1584.tif
38336/40000
topography_1585.tif
38340/40000
topography_1586.tif
38344/40000
topography_1587.tif
38348/40000
topography_1588.tif
38352/40000
topography_1589.tif
38356/40000
topography_1590.tif
38360/40000
topography_1591.tif
38364/40000
topography_1592.tif
38368/40000
topography_1593.tif
38372/40000
topography_1594.tif
38376/40000
topography_1595.tif
38380/40000
topography_1596.tif
38384/40000
topography_1597.tif
38388/40000
topography_1598.tif
38392/40000
topography_1599.tif
38396/40000
topography_1600.tif
38400/40000
topography_1601.tif
38404/40000
topography_1602.tif
38408/40000
topography_1603.tif
38412/40000
topography_1604.tif
38416/40000
topography_1605.tif
38420/40000
topography_1606.tif
38424/40000
topograp

topography_1833.tif
39332/40000
topography_1834.tif
39336/40000
topography_1835.tif
39340/40000
topography_1836.tif
39344/40000
topography_1837.tif
39348/40000
topography_1838.tif
39352/40000
topography_1839.tif
39356/40000
topography_1840.tif
39360/40000
topography_1841.tif
39364/40000
topography_1842.tif
39368/40000
topography_1843.tif
39372/40000
topography_1844.tif
39376/40000
topography_1845.tif
39380/40000
topography_1846.tif
39384/40000
topography_1847.tif
39388/40000
topography_1848.tif
39392/40000
topography_1849.tif
39396/40000
topography_1850.tif
39400/40000
topography_1851.tif
39404/40000
topography_1852.tif
39408/40000
topography_1853.tif
39412/40000
topography_1854.tif
39416/40000
topography_1855.tif
39420/40000
topography_1856.tif
39424/40000
topography_1857.tif
39428/40000
topography_1858.tif
39432/40000
topography_1859.tif
39436/40000
topography_1860.tif
39440/40000
topography_1861.tif
39444/40000
topography_1862.tif
39448/40000
topography_1863.tif
39452/40000
topograp

In [8]:
#f.close()